### Práctica de Formatos de datos: XML

El objetivo de esta práctica es crear una aplicación que gestionará información sobre las bibliotecas públicas y bibliobuses situados en la ciudad de Madrid, de titularidad nacional, comunitaria o local. 

Para ello se va a usar el catálogo "Bibliotecas y bibliobuses en la ciudad de Madrid" del portal de datos abiertos del Ayuntamiento de Madrid. Este catalogo ofrece datos de dirección, horario, servicios y coordenadas para su georeferenciación. La información se encuentra en:
https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ed35401429b83410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

Y el conjunto de datos XML que hay que procesar está situado en:
https://datos.madrid.es/egob/catalogo/201747-0-bibliobuses-bibliotecas.xml

El siguiente trozo de código permite recuperar el archivo xml en cuestion, y lo almacena en disco.

In [1]:
import urllib.request
x = urllib.request.urlopen('https://datos.madrid.es/egob/catalogo/201747-0-bibliobuses-bibliotecas.xml')
a=x.read()
a=a.decode("utf-8")
prueba=open("catalogo.xml","w")
prueba.write(str(a))
prueba.close()

HTTPError: HTTP Error 403: Forbidden

__Ejercicio 1 [5 puntos]__

Se pide mostrar al usuario un listado de los nombres de las bibliotecas que contiene el archivo con el objetivo de que elija una de las bibliotecas y se le muestre de forma amigable(sin etiquetas) por pantalla la información asociada al parque que se encuentra en el fichero recuperado:

* Nombre de la biblioteca: atributo NOMBRE
* Horario: atributo HORARIO
* Equipamiento: atributo EQUIPAMIENTO
* Transporte: atributo TRANSPORTE
* Localización: atributos CLASE-VIAL,NOMBRE-VIA,NUM,LOCALIDAD,PROVINCIA,CODIGO-POSTAL 
* Teléfono: atributo TELÉFONO
* Email: atributo EMAIL  

Observar que algunos atributos pueden no aparecer en las bibliotecas. En estos casos, simplemente se mostrará vacío o con la palabra "Desconocido"

Por ejemplo si el usuario hubiera elegido la biblioteca "Biblioteca Pública Antonio Mingote (Latina)" debería mostrarse por pantalla:

__Nombre de la biblioteca:__

Biblioteca Pública Antonio Mingote (Latina)
    
__Horario:__

De lunes a viernes de 9 a 21 horas. Sala Infantil de 15 a 21 horas. Sábados de 9 a 14 horas. Los servicios de préstamo y acceso a internet finalizan 15 minutos antes del cierre de la Biblioteca.Días de cierre: Sábados: 11 de abril; 16 de mayo; todos los sábados de julio, agosto y septiembre; 26 de diciembre. Domingos: todos. Festivos:1 y 6 de enero; 9 y 10 de abril; 1, 2 y 15 de mayo; 15 de agosto; 12 de octubre; 2 y 9 de noviembre; 7, 8, 24, 25 y 31 de diciembre.

__Equipamiento:__

184 puestos de lectura y 25 para el uso de InternetWifi gratuito
    
__Transporte:__ 

Bus: 17, 138 Cercanías Renfe: Fanjul (línea C5)

__Localización:__ 

CALLE RAFAEL FINAT, 51.MADRID. MADRID 28044

__Teléfono:__ 

915 093 625

__Email:__ 

bib.latina@madrid.org

In [ ]:
from xml.etree.ElementTree import iterparse

#Funcion que concatena todos los datos de la localizacion de la biblioteca
def localizacion(element):
    direccion = {}
    for dato in element:
        if dato.attrib["nombre"] == "NOMBRE-VIA":
            direccion["NOMBRE-VIA"] = dato.text
        if dato.attrib["nombre"] == "CLASE-VIAL":
            direccion["CLASE-VIAL"] = dato.text
        if dato.attrib["nombre"] == "NUM":
            direccion["NUM"] = dato.text
        if dato.attrib["nombre"] == "LOCALIDAD":
            direccion["LOCALIDAD"] = dato.text
        if dato.attrib["nombre"] == "PROVINCIA":
            direccion["PROVINCIA"] = dato.text
        if dato.attrib["nombre"] == "CODIGO-POSTAL":
            direccion["CODIGO-POSTAL"] = dato.text
    localizacion = direccion["CLASE-VIAL"] + " " + direccion["NOMBRE-VIA"] + ", " + direccion["NUM"] + ". " + direccion["LOCALIDAD"] + ". " + direccion["PROVINCIA"] + " " + direccion["CODIGO-POSTAL"]
    return localizacion

#Muestra por pantalla la informacion de la biblioteca seleccionada, en caso de no existir un dato, muestra "Desconocido"
def mostrarDatos(datos):
    
    print("Nombre:")
    print(datos.get("NOMBRE", "Desconocido"), "\n")
    print("HORARIO:")
    print(datos.get("HORARIO", "Desconocido"), "\n")
    print("EQUIPAMIENTO:")
    print(datos.get("EQUIPAMIENTO", "Desconocido"), "\n")
    print("TRANSPORTE:")
    print(datos.get("TRANSPORTE", "Desconocido"), "\n")
    print("LOCALIZACION:")
    print(datos.get("LOCALIZACION", "Desconocido"), "\n")
    print("TELEFONO:")
    print(datos.get("TELEFONO", "Desconocido"), "\n")
    print("EMAIL:")
    print(datos.get("EMAIL", "Desconocido"), "\n")
    

#+++++++++++++++    Creacion del diccionario   +++++++++++++++

i = 0
diccionario = {}
dictNew = {}
for (event,element) in iterparse("201747-0-bibliobuses-bibliotecas.xml",("start","end")):
    if event=="start":
        if element.tag=="atributos":
            if i > 0: #Copia los atributos de la biblioteca anterior y lo guarda en el diccionario.
                      #salta la primera iteracion ya que no hay biblioteca anterior
                diccionario[i - 1] = dictNew
            dictNew = {} #Crea un diccionario auxiliar donde se guardara los atributos de la biblioteca 
            i = i + 1
    if event=="end":
        if element.tag == "atributo":
            if element.attrib["nombre"] == "NOMBRE":
                dictNew["NOMBRE"] = element.text
            if element.attrib["nombre"] == "HORARIO":
                dictNew["HORARIO"] = element.text
            if element.attrib["nombre"] == "EQUIPAMIENTO":
                dictNew["EQUIPAMIENTO"] = element.text
            if element.attrib["nombre"] == "TRANSPORTE":
                dictNew["TRANSPORTE"] = element.text
            if element.attrib["nombre"] == "TELEFONO":
                dictNew["TELEFONO"] = element.text
            if element.attrib["nombre"] == "EMAIL":
                dictNew["EMAIL"] = element.text
            if element.attrib["nombre"] == "LOCALIZACION": #En caso de LOCALIZACION, hay que concatenar varios campos
                dictNew["LOCALIZACION"] = localizacion(element)      
                
                
#+++++++++++++++    Menu de Bibliotecas  +++++++++++++++


cont = 0
print("ELIGE LA BIBLIOTECA QUE QUIERAS CONSULTAR:")
for clave in diccionario:
    print(clave + 1, diccionario[clave]["NOMBRE"])
select = int(input())
print("\n-------------------------------------------------------------------------------------------------------------------------------\n")
print("Has seleccionado la biblioteca: ", diccionario[select - 1]["NOMBRE"], "\n")
mostrarDatos(diccionario[select - 1])
print("-------------------------------------------------------------------------------------------------------------------------------\n")
    

from xml.etree import ElementTree
f=open("201747-0-bibliobuses-bibliotecas.xml","rt")
arbol=ElementTree.parse(f)
i=1
for nodo in arbol.findall("./contenido/atributos/atributo[@nombre='NOMBRE']"):
    print(i, " ",nodo.text)
    i=i+1

__Ejercicio 2 [5 puntos]__

Este ejercicio se pide crear un buscador sobre la información recuperada, de forma que el usuario podrá buscar bibliotecas por diferentes criterios:

 * Accesibilidad
 
 * Nombre
 
 * Barrio
 
 * Distrito
 
Como resultado debería mostrarse un listado de todas las bibliotecas que cumplen las condiciones impuestas por el usuario. Se mostrará de cada biblioteca la misma información que en el caso anterior.  En caso de no existir una biblioteca con las condiciones dada, se mostrará un mensaje informativo.

Observar que habrá que preguntar al usuario por cada uno de los criterios, y éste tendrá que seleccionar un valor por cada criterio o no seleccionarlo. Después de mostrarle la información, se le volverá a preguntar si quiere seguir buscando. Entre las opciones del menú deberá existir una que sea para finalizar la búsqueda.

Normas de entrega

   * Fecha tope de entrega: 29/10/2018
   * La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosII_GrupoX donde X será el número de grupo correspondiente.


In [1]:
from xml.etree.ElementTree import iterparse

#Funcion que concatena todos los datos de la localizacion de la biblioteca
def localizacion(element):
    direccion = {}
    for dato in element:
        if dato.attrib["nombre"] == "NOMBRE-VIA":
            direccion["NOMBRE-VIA"] = dato.text
        elif dato.attrib["nombre"] == "CLASE-VIAL":
            direccion["CLASE-VIAL"] = dato.text
        elif dato.attrib["nombre"] == "NUM":
            direccion["NUM"] = dato.text
        elif dato.attrib["nombre"] == "LOCALIDAD":
            direccion["LOCALIDAD"] = dato.text
        elif dato.attrib["nombre"] == "PROVINCIA":
            direccion["PROVINCIA"] = dato.text
        elif dato.attrib["nombre"] == "CODIGO-POSTAL":
            direccion["CODIGO-POSTAL"] = dato.text
        elif dato.attrib["nombre"] == "BARRIO":
            direccion["BARRIO"] = dato.text
        elif dato.attrib["nombre"] == "DISTRITO":
            direccion["DISTRITO"] = dato.text
            
    return direccion

#Muestra por pantalla la informacion de la biblioteca seleccionada, en caso de no existir un dato, muestra "Desconocido"
def mostrarDatos(datos):
    
    print("-------------------------------------------------------------------------------------------------------")
    print("Nombre:")
    print(datos.get("NOMBRE", "Desconocido"), "\n")
    print("HORARIO:")
    print(datos.get("HORARIO", "Desconocido"), "\n")
    print("EQUIPAMIENTO:")
    print(datos.get("EQUIPAMIENTO", "Desconocido"), "\n")
    print("TRANSPORTE:")
    print(datos.get("TRANSPORTE", "Desconocido"), "\n")
    print("LOCALIZACION:")
    print(datos["LOCALIZACION"]["CLASE-VIAL"] + " " + datos["LOCALIZACION"]["NOMBRE-VIA"] + ", " + datos["LOCALIZACION"]["NUM"] + ". " +  datos["LOCALIZACION"]["LOCALIDAD"] + ". " +  datos["LOCALIZACION"]["PROVINCIA"] + " " +  datos["LOCALIZACION"]["CODIGO-POSTAL"])
    print("TELEFONO:")
    print(datos.get("TELEFONO", "Desconocido"), "\n")
    print("EMAIL:")
    print(datos.get("EMAIL", "Desconocido"), "\n")
    
def cargarDiccionario():
    diccionario = {}
    dictNew = {}
    i = 0
    for (event,element) in iterparse("201747-0-bibliobuses-bibliotecas.xml",("start","end")):
        if event=="start":
            if element.tag=="atributos":
                if i > 0: #Copia los atributos de la biblioteca anterior y lo guarda en el diccionario.
                          #salta la primera iteracion ya que no hay biblioteca anterior
                    diccionario[i - 1] = dictNew
                dictNew = {} #Crea un diccionario auxiliar donde se guardara los atributos de la biblioteca 
                i = i + 1
        if event=="end":
            if element.tag == "atributo":
                if element.attrib["nombre"] == "NOMBRE":
                    dictNew["NOMBRE"] = element.text
                if element.attrib["nombre"] == "HORARIO":
                    dictNew["HORARIO"] = element.text
                if element.attrib["nombre"] == "EQUIPAMIENTO":
                    dictNew["EQUIPAMIENTO"] = element.text
                if element.attrib["nombre"] == "TRANSPORTE":
                    dictNew["TRANSPORTE"] = element.text
                if element.attrib["nombre"] == "TELEFONO":
                    dictNew["TELEFONO"] = element.text
                if element.attrib["nombre"] == "EMAIL":
                    dictNew["EMAIL"] = element.text
                if element.attrib["nombre"] == "LOCALIZACION": #En caso de LOCALIZACION, hay que concatenar varios campos
                    dictNew["LOCALIZACION"] = localizacion(element)     
                if element.attrib["nombre"] == "ACCESIBILIDAD":
                    dictNew["ACCESIBILIDAD"] = element.text
    return diccionario

#Funcion que muestra un mensaje de control cuando no existen bibliotecas con los parámetros seleccionados
def mensajeVacio():
    print("No hay ninguna biblioteca con los parámetros requeridos")
    
#Mostrará un menú con las distintas accesibilidades y procesará la salida correspondiente
def accesibilidad(diccionario):
    
    print("0: No accesible")
    print("1: Accesible")
    print("2: Instalación parcialmente accesible para personas con movilidad reducida")
    print("3: Sin información sobre accesilibidad para personas con movilidad reducida")
    print("4: Lengua de signos")
    print("5: Señalización podotactil")
    print("6: Bucle de inducción margnético")
    print("-1: Salir al menú principal")
    opt = input()
    cont = 0
    for clave in diccionario:
        if diccionario[clave]["ACCESIBILIDAD"] == opt:
            cont = cont + 1
            mostrarDatos(diccionario[clave])      
    if cont == 0:
        mensajeVacio()
        
#Mostrará un menú con las distintas bibliotecas y procesará la salida correspondiente        
def nombre(diccionario):
    cont = 0
    print("ELIGE LA BIBLIOTECA QUE QUIERAS CONSULTAR:")
    for clave in diccionario:
        print(clave + 1, diccionario[clave]["NOMBRE"])
    select = int(input())
    print("Has seleccionado la biblioteca: ", diccionario[select - 1]["NOMBRE"], "\n")
    mostrarDatos(diccionario[select - 1])

#Mostrará un menú con las distintos barrios y procesará la salida correspondiente    
def barrio(diccionario):
    cont = 0
    barrios = []
    for clave in diccionario: #creamos una lista con todos los barrios y la mostramos por pantalla
        if diccionario[clave]["LOCALIZACION"]["BARRIO"] not in barrios:
            barrios.append(diccionario[clave]["LOCALIZACION"]["BARRIO"])
    print("Lista de todos los barrios:")
    for barrio in barrios:
        print(cont + 1, barrio)
        cont = cont + 1
    print("Elegir una opción:")
    opt = int(input())
    for clave in diccionario:
        if diccionario[clave]["LOCALIZACION"]["BARRIO"] == barrios[opt - 1]:
            cont = cont + 1
            mostrarDatos(diccionario[clave]) 
    
    
#Mostrará un menú con las distintos distritos y procesará la salida correspondiente
def distrito(diccionario):
    cont = 0
    distritos = []
    for clave in diccionario: #creamos una lista con todos los distritos y la mostramos por pantalla
        if diccionario[clave]["LOCALIZACION"]["DISTRITO"] not in distritos:
            distritos.append(diccionario[clave]["LOCALIZACION"]["DISTRITO"])
    print("Lista de todos los distritos:")
    for distrito in distritos:
        print(cont + 1, distrito)
        cont = cont + 1
    print("Elegir una opción:")
    opt = int(input())
    for clave in diccionario:
        if diccionario[clave]["LOCALIZACION"]["DISTRITO"] == distritos[opt - 1]:
            cont = cont + 1
            mostrarDatos(diccionario[clave])

#+++++++++++++++    Menu Principal  +++++++++++++++

diccionario = cargarDiccionario()
while True:
    
    print("-------------------------------------------------------------------------------------------------------")
    print("Elige una opción:")
    print("1: Accesibilidad")
    print("2: Nombre")
    print("3: Barrio")
    print("4: Distrito")
    print("0: Salir del menú")
    opcion = int(input())
    print("-------------------------------------------------------------------------------------------------------")
    if opcion == 0:
        break
    elif opcion == 1:
        accesibilidad(diccionario)
    elif opcion == 2:
        nombre(diccionario)
    elif opcion == 3:
        barrio(diccionario)
    elif opcion == 4:
        distrito(diccionario)
        

-------------------------------------------------------------------------------------------------------
Elige una opción:
1: Accesibilidad
2: Nombre
3: Barrio
4: Distrito
0: Salir del menú
1
-------------------------------------------------------------------------------------------------------
0: No accesible
1: Accesible
2: Instalación parcialmente accesible para personas con movilidad reducida
3: Sin información sobre accesilibidad para personas con movilidad reducida
4: Lengua de signos
5: Señalización podotactil
6: Bucle de inducción margnético
-1: Salir al menú principal
1
-------------------------------------------------------------------------------------------------------
Nombre:
Bibliored 

HORARIO:
Horario: de 13:30 a 20 de lunes a viernes, excepto festivos. 

EQUIPAMIENTO:
Datos de localización: Bibliored Galileo: calle Galileo, 39. Teléfono: 915 913 901.  Servicios:  Préstamo y renovación de libros.  Acceda al catálogo  para la búsqueda de libros y renovar sus préstamos. 

